# Visualizing the distribution of _P. maniculatus_ museum specimens in North America

This notebook pulls in prepared data from arctos, vertnet, and other resources (see X) & plots it on maps of North America using leaflet. These maps are interactive and easy to explore. 

In [16]:
install.packages("formattable")

Installing package into ‘/n/home11/twooldridge/apps/R’
(as ‘lib’ is unspecified)


In [17]:
library(dplyr)
library(magrittr)
library(leaflet)
library(glue)
library(htmltools)
library(stringr)
library(wesanderson)
library(repr)
library(knitr)
library(formattable)

Let's read in the data and see what it looks like (again, this is already curated from other scripts)

In [3]:
## This file is a list of samples with geocoded locations (when necessary), as well as raster data
all_samples <- read.delim("~/museum_data/samples_w_raster.txt")
head(all_samples,3)

ids,scientificname,sex,stateprovince,locality,decimallongitude,decimallatitude,preparations,lifestage,hastissue,⋯,habitat,year,total,tail,foot,ear,weight,body,ID,NA_NALCMS_LC_30m_LAEA_mmu12_urb05
UCONN:916,Peromyscus maniculatus,female,Tennessee,Cove Hardwoods along Ramsey Prong; G.S.M.N.P.,NA,NA,Skull and mandible | Skull and mandible,adult,0,⋯,NA,11,187,97,21.5,NA,22.1,NA,NA,NA
UCONN:952,Peromyscus maniculatus,male,Utah,"60 mi. W Delta, Sarcohatus; White Valley",NA,NA,Skin only | Skin only,NA,0,⋯,NA,1939,151,70,20.0,NA,NA,NA,NA,NA
UCONN:958,Peromyscus maniculatus,female,Tennessee,"hemlock, trail to trilium gop.; G.S.M.N.P.",NA,NA,Skin | Skin,adult,0,⋯,NA,NA,180,92,20.5,NA,27.7,NA,NA,NA


Let's go ahead and plot the entire dataset, before we do any filtering or anything fancy. First we'll start with grouping by long/lat, and we can scall the point size to the number of individuals at a location

In [7]:
all.plot = all_samples %>% 
    # Group by location, and whether the location is from the specimen record or geocoding
    group_by(decimallatitude,decimallongitude) %>% 
    # Add a column for the number of samples, and all the samples at the location with html line breaks
    summarise(n=n(),samples=glue_collapse(ids,sep="<br/>"))
head(all.plot,3)

decimallatitude,decimallongitude,n,samples
-26.79118,-60.44713,6,TTU:31002<br/>TTU:31003<br/>TTU:31005<br/>TTU:31004<br/>TTU:31000<br/>TTU:31001
16.96416,-96.82777,1,CAS:10986
17.21666,-97.74527,1,CAS:14235


In [52]:
pal <- colorFactor(c(wes_palette("GrandBudapest1")[2], wes_palette("GrandBudapest2")[2]), domain = c("guess", "source"))
## Make the map ##
p = leaflet(all.plot,width="100%") %>%
    addTiles(urlTemplate = "https://mts1.google.com/vt/lyrs=s&hl=en&src=app&x={x}&y={y}&z={z}&s=G", attribution = 'Google') %>%
    #addCircleMarkers(lng=~decimallongitude,lat=~decimallatitude,color=~pal(coord_origin),fillOpacity=0.7,opacity=0,radius=~log(n+5),popup = ~samples,popupOptions = popupOptions(autoPan=FALSE)) %>%
    addCircleMarkers(lng=~decimallongitude,lat=~decimallatitude,fillOpacity=0.7,opacity=0,radius=~log(n+5),popup = ~samples,popupOptions = popupOptions(autoPan=FALSE,style=list("border-color" = "rgba(0,0,0,0.5)","fontsize"="90px"))) %>%
    addLegend("bottomright", colors=c(wes_palette("GrandBudapest2")[2]), labels=c("From data"),title = "Coordinate source",opacity = 1) %>%
    addProviderTiles("CartoDB.PositronOnlyLabels") 

q=browsable(
tagList(
    tags$head(
      tags$style(
        ".leaflet-popup-content-wrapper {
        background: rgba(105,105,105,0.6);
        color: #ffffff;
        padding: 2px;
        border-radius: 8px;
        border: 2px solid rgba(0,0,0,0.8)
        }
        ",
        ".leaflet-popup-tip {
        background: transparent;
        border: none;
        box-shadow: none;
        }
        "
      )
      ),
    p
      ))


Warning message in validateCoords(lng, lat, funcName):
“Data contains 1 rows with either missing or invalid lat/lon values and will be ignored”

In [53]:
htmlwidgets::saveWidget(p, "m.html")

In [40]:
q[[2]]$height

NULL

In [11]:
## Make the map ##
q = leaflet(all.plot,width="100%") %>%
    addTiles(urlTemplate = "https://mts1.google.com/vt/lyrs=s&hl=en&src=app&x={x}&y={y}&z={z}&s=G", attribution = 'Google') %>%
    addCircleMarkers(lng=~decimallongitude,lat=~decimallatitude,color=~pal(coord_origin),fillOpacity=0.7,opacity=0,radius=~log(n+5),popup = ~samples,popupOptions = popupOptions(autoPan=FALSE)) %>%
    addLegend("bottomright", colors=c(wes_palette("GrandBudapest2")[2],wes_palette("GrandBudapest1")[2]), labels=c("From data","Geocoded (guess)"),title = "Coordinate source",opacity = 1) %>%
    addProviderTiles("CartoDB.PositronOnlyLabels") 

r = browsable(
tagList(
    tags$head(tags$style(
        ".leaflet-popup-content-wrapper {
        background: rgba(105,105,105,0.6);
        color: #ffffff;
        padding: 2px;
        border-radius: 8px;
        border: 2px solid rgba(0,0,0,0.8)
        }
        ",
        ".leaflet-popup-tip {
        background: transparent;
        border: none;
        box-shadow: none;
        }
        ")),
    q
      ))


Warning message in validateCoords(lng, lat, funcName):
“Data contains 1 rows with either missing or invalid lat/lon values and will be ignored”

ERROR: Error in pal(coord_origin): object 'coord_origin' not found


In [74]:
library(rgl)
library(rglwidget)
library(htmltools)

theta <- seq(0, 6*pi, len=100)
xyz <- cbind(sin(theta), cos(theta), theta)
lineid <- plot3d(xyz, type="l", alpha = 1:0, 
                 lwd = 5, col = "blue")["data"]

q = browsable(tagList(
  rglwidget(elementId = "example", width = 500, height = 400,
            controllers = "player"),
  playwidget("example", 
             ageControl(births = theta, ages = c(0, 0, 1),
                        objids = lineid, alpha = c(0, 1, 0)),
             start = 1, stop = 6*pi, step = 0.1, 
             rate = 6,elementId = "player")
))

class(q)

[1] "shiny.tag.list" "list"

In [75]:
class(q[[1]])
class(q[[2]])

[1] "rglWebGL"   "htmlwidget"

[1] "rglPlayer"  "htmlwidget"